<a href="https://colab.research.google.com/github/Ceciliapjr0412/COM257_jiarongPei_a4/blob/main/trainLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

输入层**  
   - 中文BERT分词器（最大长度512）。
   - 输入格式：`[CLS] + 指令文本 + [SEP]`。

2. **编码层**  
   - BERT-base（12层Transformer，768隐藏维度）。
   - 动态注意力机制捕捉方向关键词（如“右转”“东南”）。

3. **分类层**  
   - 全连接层（768维 → 标签维度）。
   - Softmax输出概率分布。

Mounted at /content/drive


In [2]:
import time
from pynvml import *

# 新增：GPU监控工具初始化
def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU内存占用: {info.used//1024**2} MB")

def print_train_time(start_time, end_time):
    total_time = end_time - start_time
    print(f"训练用时: {total_time//60} 分 {total_time%60:.2f} 秒")
# 第1：导入数据
import pandas as pd
data = {
    "instruction": ["行驶或者行走方向"] * 77,
    "input": [
        "向右急转", "往右侧打方向", "往右继续前行", "往右走", "右拐", "往右打方向", "往右侧转", "往右转", "朝右侧转", "朝右转", "右转",
        "南向东", "南朝东", "南往东", "西向南", "西朝南", "西往南", "北向西", "北朝西", "北往西", "东向北", "东朝北", "东往北",
        "东北方向往北", "东北方向朝北", "东北方朝北", "东北方往北", "东北方向北", "东北朝北", "东北往北", "东北向北",
        "东向东北", "东朝东北", "东往东北", "东向西北", "东朝西北", "东往西北",
        "东南方向往北", "东南方向朝北", "东南方朝北", "东南方往北", "东南方向北", "东南朝北", "东南往北", "东南向北",
        "西南方向往南", "西南方向朝南", "西南方朝南", "西南方往南", "西南方向南", "西南朝南", "西南往南", "西南向南",
        "西南方向往东南", "西南方向朝东南", "西南方朝东南", "西南方往东南", "西南方向东南", "西南朝东南", "西南往东南", "西南向东南",
        "西向西南", "西朝西南", "西往西南", "西向东南", "西朝东南", "西往东南",
        "西北方向往南", "西北方向朝南", "西北方朝南", "西北方往南", "西北方向南", "西北朝南", "西北往南", "西北向南",
        "往右跑", "向右跑"
    ],
    "output": ["向右转"] * 77
}
df = pd.DataFrame(data)

# 步骤3：数据预处理
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
tokenized_data = tokenizer(
    df["input"].tolist(),
    padding=True,
    truncation=True,
    return_tensors="pt"
)

# 划分训练集和测试集
train_inputs, test_inputs, train_labels, test_labels = train_test_split(
    tokenized_data["input_ids"],
    df["output"],
    test_size=0.2,
    random_state=42
)

# 步骤4：定义模型结构（基于BERT的微调模型）
from transformers import BertForSequenceClassification, TrainingArguments, Trainer

model = BertForSequenceClassification.from_pretrained(
    "bert-base-chinese",
    num_labels=1,  # 假设为二分类问题（实际需根据标签调整）
    id2label={0: "向右转", 1: "其他方向"}  # 扩展标签需修改
)

# 在步骤5前添加初始监控
print("="*40)
print("训练前资源状态:")
!nvidia-smi --query-gpu=memory.used,utilization.gpu --format=csv
print_gpu_utilization()

# 步骤5：训练模型
start_time = time.time()##添加训练时间记录
# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=8,
#     num_train_epochs=3,
#     weight_decay=0.01,
# )
# 修改训练参数添加日志记录
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',  # 新增日志目录
    logging_steps=10,      # 每10步记录一次
    report_to='none'       # 禁用wandb等集成
)
# 创建自定义回调监控资源
from transformers import TrainerCallback

class ResourceMonitorCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % args.logging_steps == 0:
            print(f"\n步骤 {state.global_step} 资源状态:")
            !nvidia-smi --query-gpu=memory.used,utilization.gpu --format=csv
            print_gpu_utilization()

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_inputs,
#     eval_dataset=test_inputs,
# )

# trainer.train()
# 修改Trainer初始化
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_inputs,
    eval_dataset=test_inputs,
    callbacks=[ResourceMonitorCallback]  # 添加回调
)

# 运行训练
trainer.train()
# 新增训练后监控
end_time = time.time()
print("\n训练后资源状态:")
!nvidia-smi --query-gpu=memory.used,utilization.gpu --format=csv
print_gpu_utilization()
print_train_time(start_time, end_time)

# 新增：持久化监控日志（保存到Google Drive）
import csv
from google.colab import drive


with open('/content/drive/monitor/training_metrics.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['Metric', 'Value'])
    writer.writerow(['Total Training Time (s)', end_time-start_time])
    writer.writerow(['Max GPU Memory Used (MB)', info.used//1024**2])

# 步骤6：评估模型
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

predictions = trainer.predict(test_inputs)
preds = np.argmax(predictions.predictions, axis=1)

print("准确率:", accuracy_score(test_labels, preds))
print("F1分数:", f1_score(test_labels, preds, average="weighted"))

# 步骤7：保存模型
model.save_pretrained("content/drive/models/direction_mode")
tokenizer.save_pretrained("content/drive/models/direction_mode")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

ValueError: You passed along `num_labels=1` with an incompatible id to label map: {0: '向右转', 1: '其他方向'}. Since those arguments are inconsistent with each other, you should remove one of them.

In [ ]:
# 步骤8：构建交互式对话框
import gradio as gr

def predict_direction(text):
    inputs = tokenizer(text, return_tensors="pt")
    outputs = model(**inputs)
    pred = np.argmax(outputs.logits.detach().numpy())
    return model.config.id2label[pred]

interface = gr.Interface(
    fn=predict_direction,
    inputs=gr.Textbox(label="输入指令"),
    outputs=gr.Textbox(label="预测方向"),
    title="行驶方向识别系统"
)

interface.launch(share=True)